In [44]:
%matplotlib inline
from __future__ import print_function
import pandas as pd
import sys
sys.path.append('/ref/analysis/pipelines/')
import kang
from collections import Counter 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [7]:
def infoparse(x):
    key   = [i.split('=')[0] for i in x.split(';')]
    value = [i.split('=')[1] for i in x.split(';')]
    return dict(zip(key,value))

In [2]:
file_refmap = '/ref/analysis/stringtie2genemodel.results/Cre/30kintron/Many_SRAs/predicted/cuffcmp.my_csv.csv.addgene.gff3.sort.gff3.refmap'
file_tmap   = '/ref/analysis/stringtie2genemodel.results/Cre/30kintron/Many_SRAs/predicted/cuffcmp.my_csv.csv.addgene.gff3.sort.gff3.tmap'
file_fa     = '/ref/analysis/References/Creinhardtii/annotation/Creinhardtii_281_v5.5.cds.fa'
file_pep    = '/ref/analysis/stringtie2genemodel.results/Cre/30kintron/Many_SRAs/finalout/my_csv.csv.addgene.gff3.sort.gff3.pep.fa'
file_cds    = '/ref/analysis/stringtie2genemodel.results/Cre/30kintron/Many_SRAs/finalout/my_csv.csv.addgene.gff3.sort.gff3.cds.fa'
file_tgff   = '/ref/analysis/stringtie2genemodel.results/Cre/30kintron/Many_SRAs/finalout/my_csv.csv.addgene.gff3.sort.gff3.merge.all.gff3.sort.gff3'
file_rgff   = '/ref/analysis/References/Creinhardtii/annotation/Creinhardtii_281_v5.5.gene.fix.gff3'

In [10]:
df_rgff      = pd.read_csv(file_rgff,sep='\t',comment='#',header=None)
mask         = (df_rgff[2] == 'mRNA')
df_rgff_mRNA = df_rgff[mask]
df_rgff_mRNA['ID'] = df_rgff_mRNA[8].apply(lambda x: infoparse(x)['ID'])
df_rgff_mRNA_ix = df_rgff_mRNA.set_index('ID')

df_tgff      = pd.read_csv(file_tgff,sep='\t',comment='#',header=None)
mask         = (df_tgff[2] == 'mRNA')
df_tgff_mRNA = df_tgff[mask]
df_tgff_mRNA['ID'] = df_tgff_mRNA[8].apply(lambda x: infoparse(x)['ID'])
df_tgff_mRNA_ix = df_tgff_mRNA.set_index('ID')

df_rgff      = pd.read_csv(file_rgff,sep='\t',comment='#',header=None)
mask         = (df_rgff[2] == 'CDS')
df_rgff_CDS = df_rgff[mask]
df_rgff_CDS['Parent'] = df_rgff_CDS[8].apply(lambda x: infoparse(x)['Parent'])
df_rgff_CDS_ix = df_rgff_CDS.set_index('Parent')

df_tgff      = pd.read_csv(file_tgff,sep='\t',comment='#',header=None)
mask         = (df_tgff[2] == 'CDS')
df_tgff_CDS = df_tgff[mask]
df_tgff_CDS['Parent'] = df_tgff_CDS[8].apply(lambda x: infoparse(x)['Parent'])
df_tgff_CDS_ix = df_tgff_CDS.set_index('Parent')

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versu

In [52]:
def get_posset(df):
    if isinstance(df,pd.Series):
        locs = set(np.arange(df[3],df[4]+1))
    else:
        locs = {0}
        for x,y in df[[3,4]].values:
            locs = locs | set(np.arange(x,y+1))
    return locs - {0}

In [85]:
def matched_genes(chromosome, qpos):
    mask            = (df_rgff_CDS[0] == chromosome) & (df_rgff_CDS[3] < max(qpos) + 100000) & (df_rgff_CDS[3] > min(qpos) - 100000) 
    df_rgff_CDS_sub = df_rgff_CDS[mask].set_index('Parent')
    matched_transcripts = []
    for ix in set(df_rgff_CDS_sub.index):
        edf        = df_rgff_CDS_sub.loc[ix]
        #chromosome = edf[0][0]
        #t_start, t_end = np.min(edf[[3,4]].values), np.max(edf[[3,4]].values)
        #if chromosome == tchr:
        tpos = get_posset(edf)
        if len(qpos & tpos) > 100 :
            matched_transcripts.append(ix)
    return matched_transcripts 

In [86]:
dic = {'qname':[], 
       'merges':[]}
tgff_posset = []
ixs = list(set(df_tgff_CDS_ix.index))
len_dist = []
for ix in tqdm(ixs):
    qpos = get_posset(df_tgff_CDS_ix.loc[ix])
    chromosome  = df_tgff_CDS_ix.loc[ix][0][0]
    matches     = matched_genes(chromosome,qpos)
    matches_set = set(['.'.join(x.split('.')[0:2]) for x in matches])
    if len(matches_set) > 1:
        #print(ix, matches_set)
        dic['qname'].append(ix)
        dic['merges'].append(matches_set)
        len_dist.append(len(matches_set))
    

100%|██████████| 25064/25064 [35:01<00:00, 12.42it/s]


In [87]:
df_merge = pd.DataFrame(dic)

In [88]:
print("number of predicted transcripts that merge reference transcripts:",len(df_merge))

number of predicted transcripts that merge reference transcripts: 545


In [91]:
sumset = {0}
for i in df_merge['merges']:
    sumset = sumset | i

In [102]:
print("number of reference trancripts that are merged by prediction",len(sumset - {0}))

number of reference trancripts that are merged by prediction 623


In [98]:
df_merge['merged genes'] = df_merge['merges'].apply(lambda x : ','.join(x))

In [100]:
df_merge.to_csv('merged.cases.txt',sep='\t')